In [1]:
import pandas as pd
import numpy as np
from math import ceil

In [2]:
teams = pd.read_csv("2021_teams.csv")
games = pd.read_csv("./games.csv")

In [3]:
numTeams = len(teams)
numGames = len(games)

In [4]:
# Create the Colley matrix

colley_matrix = 2*np.diag(np.ones(numTeams))
b = np.ones(numTeams)
k = 0


In [5]:
day_before_season = games.loc[0,'days_since_timestart'] - 1
last_day_of_season = games.loc[len(games)-1,'days_since_timestart']

In [6]:
weight_home_win = 1
weight_away_win = 1.4
weight_neutral_win = 1
segment_weights = [1,1,2,3,5]
num_segments = len(segment_weights)

In [7]:
for i in range(numGames):
  [current_day,date,team_1_id, team_1_homefield, team_1_score, team_2_id, team_2_homefield, team_2_score] = games.loc[i,:].values

  team_1_index = team_1_id - 1
  team_2_index = team_2_id - 1

  weight_index = ceil(num_segments*((current_day-day_before_season)/(last_day_of_season-day_before_season)))-1
  time_weight = segment_weights[weight_index]

  # If Team 1 won the game
  if team_1_score > team_2_score:
    # If team 1 was the home team
    if team_1_homefield == 1:
      gameWeight = weight_home_win * time_weight
    # If team 1 was the away team
    elif team_1_homefield == -1:
      gameWeight = weight_away_win * time_weight
    # If team 1 was a neutral team
    else:
      gameWeight = weight_neutral_win * time_weight
  else:
    if team_2_homefield == 1:
      gameWeight = weight_home_win * time_weight
    elif team_2_homefield == -1:
      gameWeight = weight_away_win * time_weight
    else:
      gameWeight = weight_neutral_win * time_weight

  # Update the Colley Matrix and RHS
  colley_matrix[team_1_index,team_2_index] -= gameWeight
  colley_matrix[team_2_index,team_1_index] -= gameWeight

  colley_matrix[team_1_index,team_1_index] += gameWeight
  colley_matrix[team_2_index,team_2_index] += gameWeight

  if team_1_score > team_2_score:
    b[team_1_index] += gameWeight/2
    b[team_2_index] -= gameWeight/2
  else:
    b[team_1_index] -= gameWeight/2
    b[team_2_index] += gameWeight/2

In [8]:
# Calculate the linear system
r = np.linalg.solve(colley_matrix,b)
iSort = np.argsort(-r)

In [9]:
print('\n\n************** COLLEY Rating Method **************\n')
print('===========================')
print('Rank   Rating    Team   ')
print('===========================')
if k==0:
    numberTeamToPrint = numTeams
else:
    numberTeamToPrint = k

colley_ratings = pd.DataFrame(columns=['rank','rating','team'])

for i in range(numberTeamToPrint):
    print(f'{i+1:4d}   {r[iSort[i]]:.5f}  {teams.loc[iSort[i],"team"]}')
    colley_ratings.loc[i,:] = [i+1,r[iSort[i]],teams.loc[iSort[i],"team"]]

print('')   # extra carriage return



************** COLLEY Rating Method **************

Rank   Rating    Team   
   1   1.05984  Arizona
   2   1.04588  Villanova
   3   1.04137  Tennessee
   4   1.02687  Gonzaga
   5   1.01652  Kansas
   6   1.00830  Kentucky
   7   0.99487  Boise_St
   8   0.98052  Houston
   9   0.97647  Purdue
  10   0.97058  Baylor
  11   0.96576  Arkansas
  12   0.96360  St_Mary's_CA
  13   0.96212  Auburn
  14   0.96186  Providence
  15   0.96153  UCLA
  16   0.94591  Colorado_St
  17   0.93988  Duke
  18   0.93489  San_Diego_St
  19   0.91910  Memphis
  20   0.91858  Texas_Tech
  21   0.90923  Connecticut
  22   0.90774  Iowa
  23   0.90547  Wisconsin
  24   0.88666  Illinois
  25   0.87823  Davidson
  26   0.87148  North_Carolina
  27   0.87141  Creighton
  28   0.86557  Murray_St
  29   0.86286  USC
  30   0.85028  VCU
  31   0.84502  SMU
  32   0.84231  S_Dakota_St
  33   0.83758  Seton_Hall
  34   0.83132  Virginia_Tech
  35   0.83120  Wyoming
  36   0.82576  Texas_A&M
  37   0.82469  North

In [10]:
colley_ratings.to_csv("colley_ratings.csv",index=False)